In [2]:
from pathlib import Path
import pandas as pd
import numpy as np

In [3]:
def load_dataset():
    data_path = Path("../data/results/results.csv")

    # set column names
    col_names = [
                    "type",
                    "status",
                    "price",
                    "instalment",
                    "house_name",
                    "location",
                    "bedroom_count",
                    "bathroom_count",
                    "carport_count",
                    "land_area",
                    "building_area",
                ]

    # if not available
    if not data_path.is_file():
        # TODO: if the datasets is uploaded on kaggle, add the dataset's link
        raise ReferenceError("Data kaggle is not yet available")

    return pd.read_csv(data_path, names= col_names)

In [4]:
housing = load_dataset()

In [5]:
housing.head(3)

,type,status,price,instalment,house_name,location,bedroom_count,bathroom_count,carport_count,land_area,building_area
0,Rumah,Premier,"Rp 2,1 Miliar",Cicilan: 7 Jutaan per bulan,Rumah 2 Lantai Bagus Sertifikat Hak Milik di M...,"Andir, Bandung",3,2,2,137 m²,170 m²
1,Rumah,NaN,"Rp 4,1 Miliar",Cicilan: 15 Jutaan per bulan,Rumah Cantik @ Andir (Sudirman),"Andir, Bandung",3,2,3,202 m²,300 m²
2,Rumah,NaN,"Rp 3,3 Miliar",Cicilan: 12 Jutaan per bulan,Jual Cepat Rumah di Maleber Rajawali Bisa Untu...,"Andir, Bandung",5,2,1,350 m²,258 m²


In [6]:
housing.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7611 entries, 0 to 7610
Data columns (total 11 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   type            7611 non-null   object
 1   status          145 non-null    object
 2   price           7611 non-null   object
 3   instalment      7611 non-null   object
 4   house_name      7611 non-null   object
 5   location        7611 non-null   object
 6   bedroom_count   7611 non-null   int64 
 7   bathroom_count  7611 non-null   int64 
 8   carport_count   7611 non-null   int64 
 9   land_area       7609 non-null   object
 10  building_area   7610 non-null   object
dtypes: int64(3), object(8)
memory usage: 654.2+ KB


This analysis examines a DataFrame containing 7611 rows and 11 columns related to house listings.

**Data Types:**

- Most columns (type, status, price, instalment, house_name, location) are currently stored as text (`object`) even though they might contain numerical information ("20 million").
- Numerical features (bedroom_count, bathroom_count, carport_count) have the correct data type (`int64`).
- Land and building area are numerical values stored as text (`object`) due to scraping capturing extra characters (e.g., "m²").

**Missing Values:**

- There are missing values in some columns:
    - `status` (145 missing)
    - `land_area` (2 missing)
    - `building_area` (1 missing)

**Data Cleaning Required:**

- Transform price, land_area, and building_area from text to numbers by removing characters like "Rp" and "m²".
- Decide how to handle missing values:
    - Impute missing values with appropriate strategies (e.g., median, mean) if the number of missing values is low.
    - Remove entries with missing values if they are a significant portion of the data or if imputation is not feasible.

**Additional Considerations:**

- Explore the meaning of columns like `type` and `status` and whether they require further cleaning or encoding.
- Analyze potential outliers in numerical features after cleaning.


## Data cleansing

In [7]:
# missing values
housing.isnull().sum().sort_values(ascending=0)

status            7466
land_area            2
building_area        1
type                 0
price                0
instalment           0
house_name           0
location             0
bedroom_count        0
bathroom_count       0
carport_count        0
dtype: int64

In [8]:
# drop status column
housing_clean = housing.copy()

housing_clean.drop(columns=["status", "type", "instalment"], axis=1, inplace=True)
housing_clean.columns

Index(['price', 'house_name', 'location', 'bedroom_count', 'bathroom_count',
       'carport_count', 'land_area', 'building_area'],
      dtype='object')

In [9]:
# drop rows that have land area or building area's missing
missing_area = (housing_clean["building_area"].isnull()) | (housing_clean["land_area"].isnull())
housing_clean.drop(housing_clean[missing_area].index, inplace=True)

housing_clean.head()

,price,house_name,location,bedroom_count,bathroom_count,carport_count,land_area,building_area
0,"Rp 2,1 Miliar",Rumah 2 Lantai Bagus Sertifikat Hak Milik di M...,"Andir, Bandung",3,2,2,137 m²,170 m²
1,"Rp 4,1 Miliar",Rumah Cantik @ Andir (Sudirman),"Andir, Bandung",3,2,3,202 m²,300 m²
2,"Rp 3,3 Miliar",Jual Cepat Rumah di Maleber Rajawali Bisa Untu...,"Andir, Bandung",5,2,1,350 m²,258 m²
3,Rp 580 Juta,Rumah minimalis 3 lantai di andir Bandung coco...,"Andir, Bandung",2,2,1,30 m²,80 m²
4,"Rp 1,3 Miliar",Jual Rumah Murah Bisa Dijadikan Kos”an Di Jala...,"Andir, Bandung",11,3,0,176 m²,176 m²


# Transform

In [10]:
from decimal import Decimal

def convert_price(price_str):
    try:
        clean_price = price_str.split()[1].replace(",", ".")

        if price_str.split()[2] == "Miliar":
            total_price = Decimal(clean_price) * 1_000_000_000
        elif price_str.split()[2] == "Juta":
            total_price = Decimal(clean_price) * 1_000_000
        
        return int(total_price)
    except:
        return int(0)
    
def convert_area(area_str):
    try:
        clean_area = area_str.split()[0]

        return int(clean_area)
    except:
        return int(0)
    
def clean_name(name_str):
    try:
        clean_name = name_str.replace("\r\n", "")

        return str(clean_name)
    except:
        return ""


In [11]:
housing_clean["cleaned_price"] = housing_clean["price"].apply(convert_price)
housing_clean["cleaned_land_area (m2)"] = housing_clean["land_area"].apply(convert_area)
housing_clean["cleaned_building_area (m2)"] = housing_clean["building_area"].apply(convert_area)

# clean new line in house name
housing_clean["house_name"] = housing_clean["house_name"].apply(clean_name)

In [12]:
housing_clean.head()

,price,house_name,location,bedroom_count,bathroom_count,carport_count,land_area,building_area,cleaned_price,cleaned_land_area (m2),cleaned_building_area (m2)
0,"Rp 2,1 Miliar",Rumah 2 Lantai Bagus Sertifikat Hak Milik di M...,"Andir, Bandung",3,2,2,137 m²,170 m²,2100000000,137,170
1,"Rp 4,1 Miliar",Rumah Cantik @ Andir (Sudirman),"Andir, Bandung",3,2,3,202 m²,300 m²,4100000000,202,300
2,"Rp 3,3 Miliar",Jual Cepat Rumah di Maleber Rajawali Bisa Untu...,"Andir, Bandung",5,2,1,350 m²,258 m²,3300000000,350,258
3,Rp 580 Juta,Rumah minimalis 3 lantai di andir Bandung coco...,"Andir, Bandung",2,2,1,30 m²,80 m²,580000000,30,80
4,"Rp 1,3 Miliar",Jual Rumah Murah Bisa Dijadikan Kos”an Di Jala...,"Andir, Bandung",11,3,0,176 m²,176 m²,1300000000,176,176


In [13]:
housing_clean.drop(columns=["price", "land_area", "building_area"], axis=1, inplace=True)

In [14]:
housing_clean.head(3)

,house_name,location,bedroom_count,bathroom_count,carport_count,cleaned_price,cleaned_land_area (m2),cleaned_building_area (m2)
0,Rumah 2 Lantai Bagus Sertifikat Hak Milik di M...,"Andir, Bandung",3,2,2,2100000000,137,170
1,Rumah Cantik @ Andir (Sudirman),"Andir, Bandung",3,2,3,4100000000,202,300
2,Jual Cepat Rumah di Maleber Rajawali Bisa Untu...,"Andir, Bandung",5,2,1,3300000000,350,258


In [15]:
housing_clean.rename(columns={"cleaned_price": "price", "cleaned_land_area (m2)": "land_area (m2)", "cleaned_building_area (m2)": "building_area (m2)"}, errors="raise", inplace=True)

In [16]:
housing_clean.head()

,house_name,location,bedroom_count,bathroom_count,carport_count,price,land_area (m2),building_area (m2)
0,Rumah 2 Lantai Bagus Sertifikat Hak Milik di M...,"Andir, Bandung",3,2,2,2100000000,137,170
1,Rumah Cantik @ Andir (Sudirman),"Andir, Bandung",3,2,3,4100000000,202,300
2,Jual Cepat Rumah di Maleber Rajawali Bisa Untu...,"Andir, Bandung",5,2,1,3300000000,350,258
3,Rumah minimalis 3 lantai di andir Bandung coco...,"Andir, Bandung",2,2,1,580000000,30,80
4,Jual Rumah Murah Bisa Dijadikan Kos”an Di Jala...,"Andir, Bandung",11,3,0,1300000000,176,176


In [17]:
housing_clean.isnull().sum().sort_values(ascending=0)

house_name            0
location              0
bedroom_count         0
bathroom_count        0
carport_count         0
price                 0
land_area (m2)        0
building_area (m2)    0
dtype: int64

In [18]:
housing_clean.iloc[183]

house_name            Dijual rumahSiap huni Town house  andir
location                                       Andir, Bandung
bedroom_count                                               3
bathroom_count                                              2
carport_count                                               1
price                                              1600000000
land_area (m2)                                             70
building_area (m2)                                        116
Name: 183, dtype: object

In [19]:
def export_dataset(dataset):
    data_path = Path("../data/results/results_cleaned.csv")
    
    dataset.to_csv(data_path, index=False)

export_dataset(housing_clean)